# COLLECTE ET HARMONISATION DES DONNEES - REPUBLIQUE DU BENIN

## Objectifs
   - **Thématique**: Démographie - Economie - Santé - Education - Géographie
   - **Sources**: INStad - Portails Open Data - Worl Bank API - Web scraping
   - **Couverture**: Tous les départements du bénin
   - **Période**: 2020 - 2025 (Selon la disponibilité des données au niveau des sources)
### Python version: 3.13.7

### Configuration des imports

In [109]:
# =============================
# Imports
# =============================

# Standard library
import os
import re
import json
import time
import zipfile
import warnings
import logging
from io import BytesIO
from pathlib import Path
from datetime import datetime
from typing import Optional
from urllib.parse import urljoin, urlparse
from io import StringIO

# Third-party
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

# =============================
# Configurations globales
# =============================

# Warnings
warnings.filterwarnings("ignore", category=UserWarning, module="bs4")

# Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)

# Pandas
pd.set_option("display.max_rows", 100)  # nb max de lignes affichées
pd.set_option("display.max_columns", None)  # affiche toutes les colonnes
pd.set_option("display.float_format", "{:.2f}".format)  # formatage des floats
pd.set_option("display.expand_frame_repr", False)  # évite retour ligne inutile

# Matplotlib / Seaborn
plt.style.use("seaborn-v0_8-whitegrid")  # style moderne et lisible
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12
sns.set_palette("Set2")

# =============================
# Check initialisation
# =============================

logging.info("Librairies et configurations chargées")
logging.info("Début de la collecte de données: %s", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


2025-09-20 14:07:41,932 | INFO | Librairies et configurations chargées
2025-09-20 14:07:41,934 | INFO | Début de la collecte de données: 20/09/2025 14:07:41


### Configuration des dossiers

#### Fonction de configurations de dossiers, réutilisable

In [110]:
def init_directory(base_dir: Optional[Path] = None) -> dict:
    """
    Initializes directory structure under the specified base directory.

    Creates a directory structure for storing data, including raw data,
    processed data, and final data. If directories already exist, their
    existence is logged; otherwise, they are created.

    :param base_dir: Optional base directory path. Defaults to the current
        working directory if not provided.
    :type base_dir: Optional[Path]
    :return: A dictionary with the keys 'data', 'raw_data', 'processed_data',
        and 'final_data', where each value is the Path object of the corresponding
        directory.
    :rtype: dict
    """

    if base_dir is None:
        base_dir = Path(".")

    data_dir = base_dir / "data"  # Répertoire ou se trouvera les données
    dirs = {
        "data": data_dir,
        "raw_data": data_dir / "raw_data",  # Répertoire des données brutes
        "processed_data": data_dir / "processed_data",  # Répertoire des données traitées
        "final_data": data_dir / "final_data",  # Répertoire des données finales
    }

    for name, path in dirs.items():
        try:
            if path.exists():
                logging.info(f"Dossier {name} existe déjà.")
            else:
                path.mkdir(parents=True, exist_ok=True)
                logging.info(f"Répertoire {name} prêt à {path}")
        except PermissionError as pe:
            logging.error(f"Erreur de permission pour {name}: {pe}")
        except OSError as oe:
            logging.error(f"Erreur système pour {name}: {oe}")
        except Exception as e:
            logging.error(f"Erreur inconnu pour {name}: {e}")

    return dirs

#### Initialisation des dossiers

In [111]:
DATA_DIR, RAW_DIR, PROCESSED_DIR, FINAL_DIR = init_directory().values()

logging.info("Dossiers chargés avec succès")

2025-09-20 14:07:42,007 | INFO | Dossier data existe déjà.
2025-09-20 14:07:42,008 | INFO | Dossier raw_data existe déjà.
2025-09-20 14:07:42,009 | INFO | Dossier processed_data existe déjà.
2025-09-20 14:07:42,010 | INFO | Dossier final_data existe déjà.
2025-09-20 14:07:42,011 | INFO | Dossiers chargés avec succès


### Traitement des sources

#### URL et sources API externes

In [112]:
# Base URL pour l'API de la Banque Mondiale
API_BASE_URL_WORLD_BANK = "https://api.worldbank.org/v2"

# Base URL pour l'API d'Instad (service local ou national)
API_BASE_URL_INSTAD = "https://instad.bj"

# Base URL pour l'API Overpass (OpenStreetMap) pour requêtes sur les cartes
API_BASE_URL_OPEN_STREET_MAP = "https://overpass-api.de/api/interpreter"

# Liste des URLs CSV provenant de sources externes potentielles
# Ici un exemple : données UN Data via l'API UNSD (2015-2024)
POTENTIAL_API_EXTERNAL_SOURCE_CSV_FILE = [
    "https://data.un.org/ws/rest/data/UNSD,DF_UNData_UNFCC,1.0/all/?startPeriod=2015&endPeriod=2024"
]

#### Configuration supplémentaire

In [113]:
COUNTRY_CODE = "BJ"

DEFAULT_INDICATOR_WORLD_BANK = [
    'SP.POP.TOTL',  # Population totale
    'NY.GDP.MKTP.CD',  # PIB
    'SE.PRM.NENR',  # Scolarisation primaire
    'SH.DYN.MORT',  # Taux de mortalité
    'AG.LND.TOTL.K2',  # Surface totale
    'NY.GDP.PCAP.CD',  # PIB par habitant
    'SL.TLF.TOTL.IN',  # Force de travail
    'SP.DYN.TFRT.IN'  # Taux de fertilité
]

#### Collecte WORLD BANK API

##### WorldBankAPI (mise à jour)

Cette section documente la classe WorldBankAPI, utilisée pour interroger l’API de la Banque mondiale et collecter des séries d’indicateurs pour un pays donné sur une plage d’années. La méthode principale retourne un DataFrame normalisé, prêt pour des analyses ultérieures.

- Aperçu:
  - Interroge l’API World Bank pour des indicateurs économiques, démographiques, sociaux, etc.
  - Standardise les colonnes de sortie: indicator_code, indicator_name, country_code, country_name, year, value, source, collection_date.
  - Utilise une session HTTP persistante avec en-tête User-Agent défini.

- Paramètres du constructeur (défauts actuels):
  - url: base de l’API (par défaut: https://api.worldbank.org/v2).
  - country_code: code pays ISO alpha-2 attendu par l’API (ex: BJ pour Bénin).
  - start_year, end_year: bornes temporelles (par défaut: 2015–2024).
  - default_per_page: taille de page demandée à l’API (par défaut: 100).

- Méthode principale:
  - get_indicators(indicators):
    - Entrée: liste de codes d’indicateurs (ex: SP.POP.TOTL, NY.GDP.MKTP.CD).
    - Traite chaque indicateur en appelant l’endpoint /country/{country_code}/indicator/{indicator}.
    - Sortie: pandas.DataFrame enrichi avec les métadonnées, la source (“WORLD BANK API”) et la date de collecte du jour.

- Comportements et bonnes pratiques intégrés:
  - Temporisation de 0.5s entre requêtes pour limiter la charge.
  - Conversion du champ year en numérique (coercition des valeurs invalides).
  - Journalisation informative: début de collecte par indicateur, nombre d’enregistrements récupérés, erreurs HTTP/JSON.
  - En-tête User-Agent explicite pour une identification claire côté API.

- Limites connues (inchangées):
  - La pagination n’est pas itérée au-delà de per_page (si > per_page, le surplus n’est pas récupéré).
  - Valeurs manquantes possibles (null) dans value (retournées telles quelles).
  - Gestion d’erreurs volontairement simple via logging (pas de retries/backoff).
  - Délai réseau fixe (timeout=10s) non paramétrable via le constructeur.

- Exemple rapide d’usage:
  - Instanciation: world_bank = WorldBankAPI(country_code="BJ", start_year=2015, end_year=2024)
  - Collecte: df = world_bank.get_indicators(["SP.POP.TOTL", "NY.GDP.MKTP.CD"])
  - Export: df.to_csv("data/raw_data/world_bank_data.csv", index=False)


In [114]:
class WorldBankAPI:
    def __init__(self, url: str = API_BASE_URL_WORLD_BANK, country_code: str = COUNTRY_CODE, start_year: int = 2015,
                 end_year: int = 2024, default_per_page: int = 100):
        """
        Initializes the API request session with the given parameters for interacting
        with the World Bank API. Allows setting a specific country, time range, and API
        base URL. Configures default headers for the session to include a `User-Agent`
        string customized for educational research purposes.

        :param url: The base URL of the API to be used for requests.
        :type url: str
        :param country_code: The ISO 3166-1 alpha-3 country code for the target country.
        :type country_code: str
        :param start_year: The starting year for the data range.
        :type start_year: int
        :param end_year: The ending year for the data range.
        :type end_year: int
        """
        self.url = url
        self.country_code = country_code
        self.start_year = start_year
        self.end_year = end_year
        self.per_page = default_per_page

        self.session = requests.Session()

        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Educational Research Purpose)'
        })

    def get_indicators(self, indicators: list = DEFAULT_INDICATOR_WORLD_BANK) -> pd.DataFrame:
        """
        Récupère les données pour une liste d'indicateurs économiques ou financiers via l'API World Bank.
        Les données sont accumulées dans un pandas DataFrame.

        :param indicators: Liste des codes d'indicateurs à récupérer. Si non fournie, une liste par défaut est utilisée.
        :type indicators: list
        :return: DataFrame contenant les données récupérées, avec code et nom de l'indicateur, informations pays,
                 année, valeur, source et date de collecte.
        :rtype: pandas.DataFrame
        """
        donnees = []

        for indicator in indicators:
            logging.info(f"Récupération des données pour l'indicateur {indicator}")

            url = f"{self.url}/country/{self.country_code}/indicator/{indicator}"
            params = {
                'date': f'{self.start_year}:{self.end_year}',
                'format': 'json',
                'per_page': self.per_page
            }

            try:
                response = self.session.get(url, params=params, timeout=10)
                response.raise_for_status()
                data = response.json()

                # Vérifie si des données sont disponibles
                entries = data[1] if len(data) > 1 and data[1] else []
                logging.info(f"{len(entries)} enregistrements récupérés pour {indicator} \n")

                for entry in entries:
                    donnees.append({
                        'indicator_code': entry['indicator']['id'],
                        'indicator_name': entry['indicator']['value'],
                        'country_code': entry['country']['id'],
                        'country_name': entry['country']['value'],
                        'year': entry['date'],
                        'value': entry['value']
                    })

                # Pause pour éviter de saturer l'API
                time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                logging.error(f"Erreur HTTP pour l'indicateur {indicator}: {e}")
            except ValueError as e:
                logging.error(f"Erreur JSON pour l'indicateur {indicator}: {e}")
            except Exception as e:
                logging.error(f"Erreur inattendue pour l'indicateur {indicator}: {e}")

        dataset = pd.DataFrame(donnees)

        if not dataset.empty:
            dataset['year'] = pd.to_numeric(dataset['year'], errors='coerce')
            dataset['source'] = "WORLD BANK API"
            dataset['collection_date'] = datetime.now().date()

        return dataset


##### Implémentation

In [115]:
world_bank = WorldBankAPI()
path_to_save = RAW_DIR / 'world_bank_data.csv'

has_data = False
world_bank_data = None

if not Path(path_to_save).exists():
    world_bank_data = world_bank.get_indicators()
    has_data = len(world_bank_data) > 0
    if has_data:
        logging.info(f"World Bank: {len(world_bank_data)} enrégistrement collectés")
    else:
        logging.info(f"World Bank: Aucun enrégistrement collectés")
else:
    logging.warning("Déjà éffectué")

2025-09-20 14:07:42,166 | INFO | Récupération des données pour l'indicateur SP.POP.TOTL
2025-09-20 14:07:44,502 | INFO | 10 enregistrements récupérés pour SP.POP.TOTL 

2025-09-20 14:07:45,004 | INFO | Récupération des données pour l'indicateur NY.GDP.MKTP.CD
2025-09-20 14:07:45,333 | INFO | 10 enregistrements récupérés pour NY.GDP.MKTP.CD 

2025-09-20 14:07:45,835 | INFO | Récupération des données pour l'indicateur SE.PRM.NENR
2025-09-20 14:07:46,127 | INFO | 10 enregistrements récupérés pour SE.PRM.NENR 

2025-09-20 14:07:46,630 | INFO | Récupération des données pour l'indicateur SH.DYN.MORT
2025-09-20 14:07:46,906 | INFO | 10 enregistrements récupérés pour SH.DYN.MORT 

2025-09-20 14:07:47,408 | INFO | Récupération des données pour l'indicateur AG.LND.TOTL.K2
2025-09-20 14:07:47,969 | INFO | 10 enregistrements récupérés pour AG.LND.TOTL.K2 

2025-09-20 14:07:48,474 | INFO | Récupération des données pour l'indicateur NY.GDP.PCAP.CD
2025-09-20 14:07:48,771 | INFO | 10 enregistrements 

##### Sauvegarde des données

In [116]:
if has_data and not world_bank_data.empty:
    world_bank_data.to_csv(path_to_save, index=False)

    # Taille du fichier en octets
    size_bytes = path_to_save.stat().st_size

    # Optionnel : convertir en Ko/Mo pour lecture facile
    size_kb = size_bytes / 1024
    size_mb = size_kb / 1024

    logging.info(f"Sauvegarde des données à {path_to_save} ({size_bytes} bytes / {size_kb:.2f} KB / {size_mb:.2f} MB)")
else:
    logging.warning("Déjà éffectué")

2025-09-20 14:07:50,955 | INFO | Sauvegarde des données à data\raw_data\world_bank_data.csv (7597 bytes / 7.42 KB / 0.01 MB)


#### Collecte INSTAD API - WEB SCRAPING

###### INStadScraper

Cette section documente la classe INStadScraper, utilisée pour extraire (web scraping) des tableaux HTML et des données JSON depuis le site de l’INStaD (https://instad.bj) et autres pages apparentées.

Objectif
- Automatiser la collecte de tableaux publiés sur des pages HTML (publications, indicateurs récents).
- Normaliser les résultats sous forme de DataFrame pandas afin de faciliter l’analyse et l’export.
- Enrichir les tables extraites avec des métadonnées: source_url, table_index, collection_date.

Prérequis
- Connexion Internet active.
- Respect des conditions d’utilisation du site cible.
- Bibliothèques: requests, pandas, BeautifulSoup (bs4).

Paramètres
- base_url (str): URL de base du site INStaD. Par défaut: https://instad.bj.

Méthodes
1) scrape_html_tables(urls: list, max_tables: int = 5) -> list[pd.DataFrame]
   - Rôle: Parcourt une liste d’URLs, récupère jusqu’à max_tables tableaux HTML par page.
   - Sortie: Liste de DataFrames (une par table trouvée). Colonnes ajoutées:
     - source_url: URL d’origine.
     - table_index: index du tableau sur la page (0..max_tables-1).
     - collection_date: date du jour (YYYY-MM-DD).
   - Journalisation: nombre de tableaux trouvés, taille de chaque extraction, erreurs éventuelles.

2) scrape_json_data(json_urls: list) -> list[pd.DataFrame]
   - Rôle: Appelle une liste d’URLs retournant du JSON et normalise la structure en DataFrame.
   - Sortie: Liste de DataFrames (une par ressource JSON). Colonnes ajoutées:
     - source_url, collection_date.
   - Journalisation: taille de l’extraction, erreurs HTTP/JSON.

3) scrape_demographic_data() -> list[pd.DataFrame]
   - Rôle: Cible des pages démographiques prédéfinies (indicateurs récents, publications annuelles).
   - Sortie: Liste de DataFrames issus des tableaux HTML.

4) scrape_economic_data() -> pd.DataFrame
   - Rôle: Cible des pages économiques (publications mensuelles/trimestrielles) et, le cas échéant, des endpoints JSON.
   - Sortie: DataFrame unique concaténant toutes les extractions (HTML + JSON). DataFrame vide si aucune donnée.

Bonnes pratiques intégrées
- En-tête User-Agent explicite pour éviter les blocages courants.
- Temporisation entre requêtes (1–2s) afin de limiter la charge sur le serveur.
- Gestion d’erreurs via logging (HTTP, JSON, parsing HTML).
- Métadonnées systématiques pour la traçabilité.

Limites connues
- Structure HTML susceptible de changer (sélecteurs “table” génériques).
- Tables complexes (multi-index, cellules fusionnées) parfois mal interprétées par pandas.read_html.
- Aucune stratégie de retry/backoff en cas d’échec réseau temporaire.

Exemples d’utilisation rapide
- Démographie:
  - demographic_tables = scraper.scrape_demographic_data()
  - demographic_df = pd.concat(demographic_tables, ignore_index=True) si la liste n’est pas vide.
- Économie:
  - economic_df = scraper.scrape_economic_data()
- Export:
  - df_final.to_csv("data/raw_data/instad_scraping.csv", index=False)

Conseils
- Vérifier les colonnes extraites (titres, formats numériques, dates) avant analyse.
- Conserver les champs source_url et collection_date pour l’audit et la reproductibilité.
- Adapter max_tables selon la page; augmenter au besoin si plusieurs tableaux sont attendus.


In [117]:
class INStadScraper:
    def __init__(self, base_url: str = API_BASE_URL_INSTAD):
        self.url = base_url

        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
        })

    def scrape_html_tables(self, urls: list, max_tables: int = 5):
        html_data_tables = []

        if urls:
            for url in urls:
                try:
                    response = self.session.get(url, timeout=10)
                    response.raise_for_status()
                    logging.info(f"Scraping html de l'url {response.url}")

                    soup = BeautifulSoup(response.content, 'html.parser')

                    tables = soup.find_all('table')
                    logging.info(f"{len(tables)} tableaux trouvées sur {url}")

                    for i, table in enumerate(tables[:max_tables]):
                        try:
                            dataset = pd.read_html(StringIO(str(table)))[0]
                            dataset['source_url'] = url
                            dataset['table_index'] = i
                            dataset['collection_date'] = datetime.now().date()
                            html_data_tables.append(dataset)
                            logging.info(f"Tableau {i + 1} récupéré: {dataset.shape} \n")
                        except Exception as e:
                            logging.error(f"Impossible de récupérer le table à l'index {i} sur {url}: {e}")

                    time.sleep(2)
                except requests.exceptions.RequestException as e:
                    logging.error(f"Erreur requête pour {url}: {e}")
                except Exception as e:
                    logging.error(f"Erreur inattendue pour l'url {url}: {e}")

        return html_data_tables

    def scrape_json_data(self, json_urls: list):
        json_data = []

        if json_urls:
            for url in json_urls:
                try:
                    response = self.session.get(url, timeout=10)
                    response.raise_for_status()
                    logging.info(f"Scraping json de l'url {response.url}")

                    data = response.json()

                    dataset = pd.json_normalize(data)
                    dataset['source_url'] = url
                    dataset['collection_date'] = datetime.now().date()
                    json_data.append(dataset)
                    logging.info(f"Json récupérer: {dataset.shape} \n")

                    time.sleep(1)
                except requests.exceptions.RequestException as e:
                    logging.error(f"Erreur HTTP pour {url}: {e}")
                except ValueError as e:
                    logging.error(f"JSON invalide pour {url}: {e}")
                except Exception as e:
                    logging.error(f"Erreur inattendue pour {url}: {e}")

        return json_data

    def scrape_demographic_data(self):

        urls = [
            f"{self.url}/statistiques/indicateurs-recents/43-population",  # indicateurs récents
            f"{self.url}/publications/publications-annuelles"  # publications annuelles
        ]

        return self.scrape_html_tables(urls)

    def scrape_economic_data(self):
        html_urls = [
            f"{self.url}/publications/publications-trimestrielles",  # publications trimestrielles
            f"{self.url}/publications/publications-mensuelles"  # publications mensuelles
        ]

        # A renseigner une url json a scraper
        json_urls = []

        html_data = self.scrape_html_tables(html_urls)
        json_data = self.scrape_json_data(json_urls)

        donnees = html_data + json_data
        if donnees:
            return pd.concat(donnees, ignore_index=True)
        else:
            return pd.DataFrame()


##### Implémentations

In [118]:
scraper = INStadScraper()

path_instad_to_save = RAW_DIR / 'instad_scraping.csv'

has_data = False
instad_data = None

if not Path(path_instad_to_save).exists():
    demographic_tables = scraper.scrape_demographic_data()

    if demographic_tables:
        demographic_df = pd.concat(demographic_tables, ignore_index=True)
    else:
        demographic_df = pd.DataFrame()

    logging.info(f"Nombre de lignes démographiques : {len(demographic_df)}")

    economic_df = scraper.scrape_economic_data()
    logging.info(f"Nombre de lignes économiques : {len(economic_df)}")

    instad_data = pd.concat([demographic_df, economic_df], ignore_index=True)
    has_data = len(instad_data) > 0
    logging.info(f"Total lignes récupérées : {len(instad_data)}")
else:
    logging.warning("Déjà éffectué")


2025-09-20 14:07:53,719 | INFO | Scraping html de l'url https://instad.bj/statistiques/indicateurs-recents/43-population
2025-09-20 14:07:53,827 | INFO | 4 tableaux trouvées sur https://instad.bj/statistiques/indicateurs-recents/43-population
2025-09-20 14:07:53,838 | INFO | Tableau 1 récupéré: (1, 8) 

2025-09-20 14:07:53,851 | INFO | Tableau 2 récupéré: (13, 5) 

2025-09-20 14:07:53,858 | INFO | Tableau 3 récupéré: (5, 10) 

2025-09-20 14:07:53,868 | INFO | Tableau 4 récupéré: (5, 10) 

2025-09-20 14:07:57,760 | INFO | Scraping html de l'url https://instad.bj/publications/publications-annuelles
2025-09-20 14:07:57,969 | INFO | 7 tableaux trouvées sur https://instad.bj/publications/publications-annuelles
2025-09-20 14:07:57,975 | INFO | Tableau 1 récupéré: (1, 8) 

2025-09-20 14:07:57,986 | INFO | Tableau 2 récupéré: (11, 6) 

2025-09-20 14:07:58,014 | INFO | Tableau 3 récupéré: (107, 6) 

2025-09-20 14:07:58,023 | INFO | Tableau 4 récupéré: (27, 6) 

2025-09-20 14:07:58,029 | INFO | 

##### Sauvegarde des données

In [119]:
if has_data and not instad_data.empty:
    instad_data.to_csv(path_instad_to_save, index=False)

    # Taille du fichier en octets
    size_instad_bytes = path_instad_to_save.stat().st_size

    # Optionnel : convertir en Ko/Mo pour lecture facile
    size_instad_kb = size_instad_bytes / 1024
    size_instad_mb = size_instad_kb / 1024

    logging.info(
        f"Sauvegarde des données à {path_instad_to_save} ({size_instad_bytes} bytes / {size_instad_kb:.2f} KB / {size_instad_mb:.2f} MB)")
else:
    logging.warning("Déjà éffectué")

2025-09-20 14:08:12,134 | INFO | Sauvegarde des données à data\raw_data\instad_scraping.csv (31837 bytes / 31.09 KB / 0.03 MB)
